In [0]:
%run ../utlis

In [0]:
%run ../common

In [0]:
def main():
    try:
        last_read_timestamp = read_last_timestamp(watermark_table)
        new_data_df = read_new_data(jdbcUrl, jdbcUsername, jdbcPassword, source_table, last_read_timestamp)
        if new_data_df.count() > 0:
            # Display the new data
            display(new_data_df)
            
            # Get the latest timestamp from the new data
            latest_timestamp = new_data_df.agg({"last_updated": "max"}).collect()[0][0]
            
            # Update the watermark table with the latest timestamp
            update_watermark_table(watermark_table, latest_timestamp)
            
            # Write the new data to the target table
            new_data_df.write.format("delta").mode("append").saveAsTable(target_table)
        else:
            logger.info("No new data to process.")
    except Exception as e:
        logger.error(f"Error in main function: {e}")
        raise
        

In [0]:
if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO)
    logger = logging.getLogger(__name__)
    watermark_table = "deloitte.sudhanshu.watermark_data"
    source_table = "dbo.raj_practice"
    target_table = "deloitte.sudhanshu.raw_sqlserver_data"

    main()